In [1]:
# default_exp core

# Core Functions
> Python library to access online data from the [Joshua Project](https://joshuaproject.net/) via [APIv2](https://joshuaproject.net/resources/datasets)

In [2]:
# export
import urllib3
from urllib3.util.url import Url
import certifi
import json
from functools import partial
from datetime import date

In [3]:
#hide
from nbdev.showdoc import *

### Generic Query Function:

In [4]:
# hide
api_key='w3NOS49TW7fg' #'get_your_key'

In [5]:
# export
def url_path_query(path:str,cntry:str=None,pgid:str=None,extra=None,month=None,day=None)->str:
    """Builds and returns a string url to query `path` for `cntry` 
    with optional `pgid`,`extra`,`month` and `day`."""
    query = ''
    if cntry is not None: query = query+'ROG3='+cntry
    if pgid  is not None: query = query+'&PeopleID3='+pgid
    if extra is not None: query = query+'&'+extra
    if 'upgotd' in path:
        today = date.today()
        if day is None:   day   = today.strftime("%-d")
        if month is None: month = today.strftime("%-m")
        query = query+'&LRofTheDayMonth='+str(month) 
        query = query+'&LRofTheDayDay='  +str(day)
    query = query+'&api_key='+api_key
    return Url(scheme='https', host='joshuaproject.net', path=path, query=query).url

In [34]:
#show_doc(url_path_query)

In [7]:
# export
url_pgs_cntry  = partial(url_path_query,'/api/v2/people_groups')
url_pgs_cntry.__doc__ = """Get all people groups in a specific country."""

url_pg_cntry   = partial(url_pgs_cntry)
url_pg_cntry.__doc__ = """Get a specific people group in a specific country."""

url_upgs_cntry = partial(url_pgs_cntry,extra='LeastReached=Y')
url_upgs_cntry.__doc__ = """Get all unreached people groups in a specific country."""

url_upgotd     = partial(url_path_query,'/api/v2/upgotd')
url_upgotd.__doc__ = """Get data for the UPG of the day."""

In [31]:
url_upgotd()

'https://joshuaproject.net/api/v2/upgotd?&LRofTheDayMonth=1&LRofTheDayDay=23&api_key=w3NOS49TW7fg'

In [32]:
url_upgotd(day=10,month=2)

'https://joshuaproject.net/api/v2/upgotd?&LRofTheDayMonth=2&LRofTheDayDay=10&api_key=w3NOS49TW7fg'

Functions to query people group(s):

In [12]:
url_upgs_cntry('IN')

'https://joshuaproject.net/api/v2/people_groups?ROG3=IN&LeastReached=Y&api_key=w3NOS49TW7fg'

In [13]:
# assert  url_pgs_cntry('EN') == 'http://joshuaproject.net/api/v2/people_groups?ROG3=EN&api_key=w3NOS49TW7fg'
# assert url_upgs_cntry('EN') == 'http://joshuaproject.net/api/v2/people_groups?ROG3=EN&LeastReached=Y&api_key=w3NOS49TW7fg'

## Testing

In [14]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

In [15]:
url = url_pgs_cntry('RS')
url

'https://joshuaproject.net/api/v2/people_groups?ROG3=RS&api_key=w3NOS49TW7fg'

In [16]:
resp = http.request('GET',url)

In [17]:
if resp.status != 200:
    resp_j = json.loads(resp.data)
    s = resp_j['status']['status_code']
    m = resp_j['status']['message']
    print(f'{s}: {m}')


In [18]:
url = url_pg_cntry('IN','16152')
url

'https://joshuaproject.net/api/v2/people_groups?ROG3=IN&PeopleID3=16152&api_key=w3NOS49TW7fg'

In [19]:
resp_j = json.loads(resp.data)

In [20]:
type(resp.data)

bytes

In [21]:
print(json.dumps(resp_j['data'], sort_keys=False, indent=2))

[
  {
    "PeopleID3ROG3": "10125RS",
    "ROG3": "RS",
    "Ctry": "Russia",
    "CtryShort": "Russia",
    "CtryLiteracyRate": 100,
    "CtryLiteracySource": "Ethnologue 2018",
    "PeopleID3": 10125,
    "ROP3": 100011,
    "PeopNameInCountry": "Abaza",
    "PeopNameAcrossCountries": "Abaza",
    "Population": 45000,
    "ROG2": "ASI",
    "Continent": "Asia",
    "RegionCode": 9,
    "RegionName": "Europe, Eastern and Eurasia",
    "10_40Window": "N",
    "LocationInCountry": "Karachayevo-Cherkesiya and Stavropol\u2019skiy Kray.",
    "PeopleID1": 12,
    "AffinityBloc": "Eurasian Peoples",
    "PeopleID2": 157,
    "PeopleCluster": "Caucasus",
    "JPScalePC": 2,
    "PopulationPGAC": 73000,
    "JPScalePGAC": 1,
    "JPScale": 1,
    "JPScaleTemp": null,
    "LeastReached": "Y",
    "Frontier": "Y",
    "WorkersNeeded": 1,
    "Subgroups": null,
    "ROL3": "abq",
    "PrimaryLanguageName": "Abaza",
    "ROL4": 0,
    "PrimaryLanguageDialect": null,
    "Written": "Y",
    "Bible

In [22]:
resp_j['meta']['pagination']

{'total_count': 180, 'total_pages': 2, 'current_page': 1, 'limit': 100}

In [23]:
# TODO: Add procedding for multi-page results
# for i in range(resp_j['meta']['pagination']['total_count']):
#     print(resp_j['data'][i]['PeopNameInCountry'])

In [24]:
resp_j['data'][0]

{'PeopleID3ROG3': '10125RS',
 'ROG3': 'RS',
 'Ctry': 'Russia',
 'CtryShort': 'Russia',
 'CtryLiteracyRate': 100,
 'CtryLiteracySource': 'Ethnologue 2018',
 'PeopleID3': 10125,
 'ROP3': 100011,
 'PeopNameInCountry': 'Abaza',
 'PeopNameAcrossCountries': 'Abaza',
 'Population': 45000,
 'ROG2': 'ASI',
 'Continent': 'Asia',
 'RegionCode': 9,
 'RegionName': 'Europe, Eastern and Eurasia',
 '10_40Window': 'N',
 'LocationInCountry': 'Karachayevo-Cherkesiya and Stavropol’skiy Kray.',
 'PeopleID1': 12,
 'AffinityBloc': 'Eurasian Peoples',
 'PeopleID2': 157,
 'PeopleCluster': 'Caucasus',
 'JPScalePC': 2,
 'PopulationPGAC': 73000,
 'JPScalePGAC': 1,
 'JPScale': 1,
 'JPScaleTemp': None,
 'LeastReached': 'Y',
 'Frontier': 'Y',
 'WorkersNeeded': 1,
 'Subgroups': None,
 'ROL3': 'abq',
 'PrimaryLanguageName': 'Abaza',
 'ROL4': 0,
 'PrimaryLanguageDialect': None,
 'Written': 'Y',
 'BibleStatus': 0,
 'NumberLanguagesSpoken': 1,
 'RLG3': 6,
 'PrimaryReligion': 'Islam',
 'RLG4': None,
 'ReligionSubdivision'

In [25]:
import pandas as pd

### Convert JSON into a Pandas DataFrame

In [26]:
pd.DataFrame(resp_j['data'])

,PeopleID3ROG3,ROG3,Ctry,CtryShort,CtryLiteracyRate,CtryLiteracySource,PeopleID3,ROP3,PeopNameInCountry,PeopNameAcrossCountries,...,MapCreditURL,MapCopyright,GSEC,JF,AudioRecordings,NTOnline,GospelRadio,PEID,Longitude,Latitude
0,10125RS,RS,Russia,Russia,100,Ethnologue 2018,10125,100011,Abaza,Abaza,...,None,N,1,N,N,,N,48739.0,41.889520,44.221680
1,10130RS,RS,Russia,Russia,100,Ethnologue 2018,10130,100018,Abkhaz,Abkhaz,...,http://en.wikipedia.org/wiki/File:Caucasus_eth...,N,,Y,N,,N,NaN,39.966281,43.412621
2,10161RS,RS,Russia,Russia,100,Ethnologue 2018,10161,100079,"Adyghe, West Circassian",Adyghe,...,None,N,3,Y,Y,,N,24434.0,40.113917,44.594719
3,10184RS,RS,Russia,Russia,100,Ethnologue 2018,10184,100126,Aghul,Aghul,...,None,Y,1,N,Y,,N,24403.0,47.554020,41.930850
4,10224RS,RS,Russia,Russia,100,Ethnologue 2018,10224,100187,Akhvakh,Akhvakh,...,None,N,0,N,N,,N,24405.0,46.326160,42.384760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,12877RS,RS,Russia,Russia,100,Ethnologue 2018,12877,105458,"Kurd, Kurmanji","Kurd, Kurmanji",...,,N,4,Y,Y,KMRIBT,N,24465.0,39.732964,43.613956
96,12933RS,RS,Russia,Russia,100,Ethnologue 2018,12933,105550,Kyrgyz,Kyrgyz,...,None,N,,Y,Y,KIRBSKN2DA,N,NaN,37.564658,55.699060
97,12952RS,RS,Russia,Russia,100,Ethnologue 2018,12952,105583,Lak,Lak,...,None,Y,1,Y,N,,N,24467.0,46.146480,44.374500
98,13007RS,RS,Russia,Russia,100,Ethnologue 2018,13007,105674,"Latvian, Lett",Latvian,...,,N,1,Y,Y,LATBSL,N,48826.0,28.349894,57.818685


In [27]:
json_str = json.dumps(resp_j['data'])

In [28]:
json_str

'[{"PeopleID3ROG3": "10125RS", "ROG3": "RS", "Ctry": "Russia", "CtryShort": "Russia", "CtryLiteracyRate": 100, "CtryLiteracySource": "Ethnologue 2018", "PeopleID3": 10125, "ROP3": 100011, "PeopNameInCountry": "Abaza", "PeopNameAcrossCountries": "Abaza", "Population": 45000, "ROG2": "ASI", "Continent": "Asia", "RegionCode": 9, "RegionName": "Europe, Eastern and Eurasia", "10_40Window": "N", "LocationInCountry": "Karachayevo-Cherkesiya and Stavropol\\u2019skiy Kray.", "PeopleID1": 12, "AffinityBloc": "Eurasian Peoples", "PeopleID2": 157, "PeopleCluster": "Caucasus", "JPScalePC": 2, "PopulationPGAC": 73000, "JPScalePGAC": 1, "JPScale": 1, "JPScaleTemp": null, "LeastReached": "Y", "Frontier": "Y", "WorkersNeeded": 1, "Subgroups": null, "ROL3": "abq", "PrimaryLanguageName": "Abaza", "ROL4": 0, "PrimaryLanguageDialect": null, "Written": "Y", "BibleStatus": 0, "NumberLanguagesSpoken": 1, "RLG3": 6, "PrimaryReligion": "Islam", "RLG4": null, "ReligionSubdivision": null, "PercentAdherents": 0.1,

In [29]:
resp.data

b'{"meta":{"pagination":{"total_count":180,"total_pages":2,"current_page":1,"limit":100}},"data":[{"PeopleID3ROG3":"10125RS","ROG3":"RS","Ctry":"Russia","CtryShort":"Russia","CtryLiteracyRate":100,"CtryLiteracySource":"Ethnologue 2018","PeopleID3":10125,"ROP3":100011,"PeopNameInCountry":"Abaza","PeopNameAcrossCountries":"Abaza","Population":45000,"ROG2":"ASI","Continent":"Asia","RegionCode":9,"RegionName":"Europe, Eastern and Eurasia","10_40Window":"N","LocationInCountry":"Karachayevo-Cherkesiya and Stavropol\\u2019skiy Kray.","PeopleID1":12,"AffinityBloc":"Eurasian Peoples","PeopleID2":157,"PeopleCluster":"Caucasus","JPScalePC":2,"PopulationPGAC":73000,"JPScalePGAC":1,"JPScale":1,"JPScaleTemp":null,"LeastReached":"Y","Frontier":"Y","WorkersNeeded":1,"Subgroups":null,"ROL3":"abq","PrimaryLanguageName":"Abaza","ROL4":0,"PrimaryLanguageDialect":null,"Written":"Y","BibleStatus":0,"NumberLanguagesSpoken":1,"RLG3":6,"PrimaryReligion":"Islam","RLG4":null,"ReligionSubdivision":null,"PercentAd

All people groups in a specific country

http://joshuaproject.net/api/v2/people_groups?ROG3=YM&api_key=aWye9lV20QtF

All unreached people groups in a specific country

https://joshuaproject.net/api/v2/people_groups?ROG3=YM&LeastReached=Y&api_key=aWye9lV20QtF

A people group in a specific country

http://joshuaproject.net/api/v2/people_groups?ROG3=AF&PeopleID3=19409&api_key=aWye9lV20QtF

All countries a specific people group lives in

http://joshuaproject.net/api/v2/people_groups?PeopleID3=10294&api_key=aWye9lV20QtF

**Country summary data**

http://joshuaproject.net/api/v2/countries?ROG3=AF&api_key=aWye9lV20QtF

**Language summary info**

http://joshuaproject.net/api/v2/languages?ROL3=hau&api_key=aWye9lV20QtF

**Unreached of the Day** (basic data October 16th)

https://joshuaproject.net/api/v2/upgotd?api_key=aWye9lV20QtF&ROL3Profile=eng&LRofTheDayMonth=10&LRofTheDayDay=16

## Export -

In [33]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
